# Decision Trees from Scratch

In [769]:
import numpy as np

In [770]:
data = np.array([
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
])



In [771]:
data

array([['12.0', '1.5', '1', 'Wine'],
       ['5.0', '2.0', '0', 'Beer'],
       ['40.0', '0.0', '1', 'Whiskey'],
       ['13.5', '1.2', '1', 'Wine'],
       ['4.5', '1.8', '0', 'Beer'],
       ['38.0', '0.1', '1', 'Whiskey'],
       ['11.5', '1.7', '1', 'Wine'],
       ['5.5', '2.3', '0', 'Beer']], dtype='<U32')

# Encode the Dataset

In [772]:
X= data[:,:-1].astype(np.float32)
y = data[:,-1]

label_encoding = {'Beer':0,'Wine':1,'Whiskey':2}
label_decoding = {0:'Beer',1:'Wine',2:'Whiskey'}
for i  in range(len(y)):
    y[i] = label_encoding[y[i]]

y = y.astype(np.int32)



# Gini Impurity


In [773]:
def giniImpurity(labels):
    n = np.size(labels)
    _,counts = np.unique(labels,return_counts=True)
    probabilites = counts/n

    sum = 0
    for p_i in probabilites:
        sum += p_i **2

    return 1 -sum





# Best Split Finder

In [774]:
def bestSplitFinder(X,y):

   features_min_impurities = []
   features_corresponding_thresholds = []


   for col in range(X.shape[1]):
      vals = X[:,col]
      unique_vals = np.unique(vals)
      n = np.size(unique_vals)
      thresholds=[]
      if(np.size(unique_vals)>1):
         thresholds = [(unique_vals[i] + unique_vals[i+1])/2 for i in range(n-1)]
      else:
         thresholds  = unique_vals



      res ={}
      for threshold in thresholds:

         yes =[]
         no =[]
         for index,val in enumerate(vals):
            if val > threshold:
               yes.append(index)
            else:
               no.append(index)



         yes_len = np.size(yes)
         labels_yes = y[yes]
         impurity_yes = giniImpurity(labels_yes)


       

         no_len = np.size(no)
         labels_no = y[no]
         impurity_no = giniImpurity(labels_no)

        


         avg_impurity = (yes_len * impurity_yes  + no_len * impurity_no)/(yes_len + no_len)

       
         res[threshold] = avg_impurity
       
      res_thresh_with_min_impurity = min(res,key=res.get)
      res_min_impurity = res[res_thresh_with_min_impurity]

      features_min_impurities.append(res_min_impurity)
      features_corresponding_thresholds.append(res_thresh_with_min_impurity)

   res_feature = np.argmin(features_min_impurities)
   res_threshold = features_corresponding_thresholds[res_feature]

   return res_feature,res_threshold,features_min_impurities[res_feature]

In [775]:
res_feature, res_threshold, best_gini = bestSplitFinder(X,y)
print(res_feature)
print(res_threshold)
print(best_gini)

0
8.5
0.3


# Decision Tree Node

In [776]:
class Node:
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value =value

    def is_leaf(self):
        return self.left == None and self.right == None
    
    






# Building the Decision Tree

In [777]:

def build(features,labels,cur_depth,max_depth =100):
    if len(np.unique(labels) )==1  or cur_depth == max_depth :
        leaf_value = np.bincount(labels).argmax()
        return Node(value = leaf_value)
    

    best_feature, best_threshold,_ = bestSplitFinder(features,labels)

   


    vals = features[:,best_feature]
    yes =[]
    no =[]
    for index,val in enumerate(vals):

       if val > best_threshold:
          yes.append(index)
       else:
          no.append(index)


    features_left = np.array(features[yes])
    features_right = np.array(features[no])


    labels_left = np.array(labels[yes])
    labels_right = np.array(labels[no])







    left_tree = build(features_left,labels_left,cur_depth+1,max_depth)
    right_tree = build(features_right,labels_right,cur_depth+1,max_depth)

    return Node(best_feature,best_threshold,left_tree,right_tree)


    


    
    

In [778]:
max_depth = 100
tree_root = build(X,y,0,max_depth)

# Printing the Decision Tree

In [779]:
features = ['Alchohol Content','Sugar','Color']
root = tree_root
depth = 0
# node_n0 =1

def printDecisionTree(node, depth=0):
    indent = "  " * depth  # two spaces per depth level for clarity

    if node.is_leaf():
        print(f"{indent}Predict: {label_decoding[node.value]} ")
        return

    feature = features[node.feature_index]
    threshold = node.threshold

    print(f"{indent}if {feature} > {threshold:.2f}:")
    printDecisionTree(node.left, depth + 1)
    
    print(f"{indent}else:  # {feature} <= {threshold:.2f}")
    printDecisionTree(node.right, depth + 1)
    


printDecisionTree(root,0)


if Alchohol Content > 8.50:
  if Alchohol Content > 25.75:
    Predict: Whiskey 
  else:  # Alchohol Content <= 25.75
    Predict: Wine 
else:  # Alchohol Content <= 8.50
  Predict: Beer 


# Testing 

In [780]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])


predictions = []

for test_x in test_data:
    root = tree_root
    depth =0
    while not root.is_leaf():
        print(f'depth {depth}  Feature {root.feature_index}   Threshold {root.threshold}')
        depth = depth +1
        if test_x[root.feature_index] > root.threshold:
            root = root.left
        else:
            root = root.right

    print(root.value)
    predictions.append(label_decoding[root.value])

depth 0  Feature 0   Threshold 8.5
0
depth 0  Feature 0   Threshold 8.5
depth 1  Feature 0   Threshold 25.75
2
depth 0  Feature 0   Threshold 8.5
depth 1  Feature 0   Threshold 25.75
1


In [781]:
print(predictions)

['Beer', 'Whiskey', 'Wine']


# Decsion Tree Class

In [782]:
import numpy as np

class DecisionTree:





    def __init__(self,max_depth=5):
        self.max_depth = max_depth
        self.root = None
    

    class Node:
        def __init__(self,feature_index=None,threshold=None,left=None,right=None,value=None):
            self.feature_index = feature_index
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value =value
    
        def is_leaf(self):
            return self.left == None and self.right == None
    

    def fit(self,X,y):
        self.root= self._build(X,y,0,self.max_depth)
    

    def _giniImpurity(self,labels):
        n = np.size(labels)
        _,counts = np.unique(labels,return_counts=True)
        probabilites = counts/n

        sum = 0
        for p_i in probabilites:
            sum += p_i **2

        return 1 -sum

    def _bestSplitFinder(self,X,y):
        features_min_impurities = []
        features_corresponding_thresholds = []


        for col in range(X.shape[1]):
           vals = X[:,col]
           unique_vals = np.unique(vals)
           n = np.size(unique_vals)
           thresholds=[]
           if(np.size(unique_vals)>1):
              thresholds = [(unique_vals[i] + unique_vals[i+1])/2 for i in range(n-1)]
           else:
              thresholds  = unique_vals



           res ={}
           for threshold in thresholds:

              yes =[]
              no =[]
              for index,val in enumerate(vals):
                 if val > threshold:
                    yes.append(index)
                 else:
                    no.append(index)



              yes_len = np.size(yes)
              labels_yes = y[yes]
              impurity_yes =   self._giniImpurity(labels_yes)




              no_len = np.size(no)
              labels_no = y[no]
              impurity_no = self._giniImpurity(labels_no)




              avg_impurity = (yes_len * impurity_yes  + no_len * impurity_no)/(yes_len + no_len)


              res[threshold] = avg_impurity

           res_thresh_with_min_impurity = min(res,key=res.get)
           res_min_impurity = res[res_thresh_with_min_impurity]

           features_min_impurities.append(res_min_impurity)
           features_corresponding_thresholds.append(res_thresh_with_min_impurity)

        res_feature = np.argmin(features_min_impurities)
        res_threshold = features_corresponding_thresholds[res_feature]

        return res_feature,res_threshold,features_min_impurities[res_feature]

    def _build(self,features,labels,cur_depth,max_depth =100):
        if len(np.unique(labels) )==1  or cur_depth == max_depth :
            leaf_value = np.bincount(labels).argmax()
            return Node(value = leaf_value)


        best_feature, best_threshold,_ = self._bestSplitFinder(features,labels)

    


        vals = features[:,best_feature]
        yes =[]
        no =[]
        for index,val in enumerate(vals):

           if val > best_threshold:
              yes.append(index)
           else:
              no.append(index)


        features_left = np.array(features[yes])
        features_right = np.array(features[no])


        labels_left = np.array(labels[yes])
        labels_right = np.array(labels[no])







        left_tree = self._build(features_left,labels_left,cur_depth+1,max_depth)
        right_tree = self._build(features_right,labels_right,cur_depth+1,max_depth)

        return Node(best_feature,best_threshold,left_tree,right_tree)
    

    def predict(self,X):
        return [self.predict_one(x,self.root) for x in X]


    def predict_one(self,x,root):
       
        
        # depth =0
        while not root.is_leaf():
            # print(f'depth {depth}  Feature {root.feature_index}   Threshold {root.threshold}')
            # depth = depth +1
            if x[root.feature_index] > root.threshold:
                root = root.left
            else:
                root = root.right

        return root.value
    
    def printDecsionTree(self,features,labels):
        self._printDecisionTree(self.root,features,labels,0)
        
    def _printDecisionTree(self,node, features,labels,depth=0):
        indent = "  " * depth  # two spaces per depth level for clarity

        if node.is_leaf():
            print(f"{indent}Predict: {labels[node.value]} ")
            return

        feature = features[node.feature_index]
        threshold = node.threshold

        print(f"{indent}if {feature} > {threshold:.2f}:")
        self._printDecisionTree(node.left,features,labels, depth + 1)

        print(f"{indent}else:  # {feature} <= {threshold:.2f}")
        self._printDecisionTree(node.right,features,labels, depth + 1)

  

    
    
    
    
    

In [783]:
myDecisionTree = DecisionTree(5)
myDecisionTree.fit(X,y)
precitions = myDecisionTree.predict(test_data)
for x in precitions:
    print(label_decoding[x])



Beer
Whiskey
Wine


In [784]:
myDecisionTree.printDecsionTree(features,["Beer","Wine","Whiskey"])

if Alchohol Content > 8.50:
  if Alchohol Content > 25.75:
    Predict: Whiskey 
  else:  # Alchohol Content <= 25.75
    Predict: Wine 
else:  # Alchohol Content <= 8.50
  Predict: Beer 


# Testing on some famous Classification Dataset

In [785]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

In [786]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [787]:
## Dividing the dataset into train and test

n = len(X)
n_train = int(0.80 * n)

X_train = X[0:n_train,:]
y_train  = y[0:n_train]

X_test = X[n_train:,:]
y_test =y[n_train:]

In [788]:
myDecisionTree = DecisionTree(40)
myDecisionTree.fit(X_train,y_train)
predictions  = myDecisionTree.predict(X_test)


In [789]:
def accuracy(y,y_hat):
    return np.sum(y==y_hat)/len(y)

In [790]:
print(accuracy(y_test,predictions))

0.7333333333333333


In [791]:
features = ['sepal length','sepal width', 'petal length', 'petal width']
labels= ['Iris setosa', 'Iris versicolor', 'Iris virginica']
myDecisionTree.printDecsionTree(features,labels)

if petal length > 2.45:
  if petal length > 4.95:
    if petal width > 1.75:
      Predict: Iris virginica 
    else:  # petal width <= 1.75
      if sepal width > 2.45:
        Predict: Iris versicolor 
      else:  # sepal width <= 2.45
        Predict: Iris virginica 
  else:  # petal length <= 4.95
    if sepal length > 4.95:
      Predict: Iris versicolor 
    else:  # sepal length <= 4.95
      if sepal width > 2.45:
        Predict: Iris virginica 
      else:  # sepal width <= 2.45
        Predict: Iris versicolor 
else:  # petal length <= 2.45
  Predict: Iris setosa 
